In [267]:
import sys
import os
import numpy as np
import pandas as pd
import math
import sklearn
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import pdb
import nba_py as nba
from dateutil import parser
from sklearn import model_selection,preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
pd.options.display.max_columns = None

In [238]:
filenames = ['boxscores_spreads_cleaned.csv']
filename = filenames[0]
df = pd.read_csv(filename)
df.drop(['Unnamed: 0'],axis=1,inplace=True)
display(df)


,GAME_ID,date,team,home,away,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,AST,PF,STL,TO,BLK,PTS,spread,PLUS_MINUS,ou,total
0,21300002,2013-10-29,Miami Heat,1,0,37.0,72.0,0.514,11.0,20.0,0.550,22.0,29.0,5.0,35.0,26.0,21.0,10.0,18.0,7.0,107.0,-5.0,12.0,O,188.0
1,21300002,2013-10-29,Chicago Bulls,0,1,35.0,83.0,0.422,7.0,26.0,0.269,18.0,23.0,11.0,30.0,23.0,27.0,11.0,18.0,4.0,95.0,5.0,-12.0,O,188.0
2,21300003,2013-10-29,Los Angeles Lakers,1,0,42.0,93.0,0.452,14.0,29.0,0.483,18.0,28.0,18.0,34.0,23.0,23.0,8.0,19.0,6.0,116.0,9.5,13.0,O,201.5
3,21300003,2013-10-29,Los Angeles Clippers,0,1,41.0,83.0,0.494,8.0,21.0,0.381,13.0,23.0,10.0,30.0,27.0,21.0,11.0,16.0,4.0,103.0,-9.5,-13.0,O,201.5
4,21300001,2013-10-29,Orlando Magic,0,1,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,13.0,26.0,17.0,26.0,10.0,17.0,6.0,87.0,12.0,-10.0,U,189.5
5,21300001,2013-10-29,Indiana Pacers,1,0,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,10.0,34.0,17.0,13.0,4.0,20.0,18.0,97.0,-12.0,10.0,U,189.5
6,21300010,2013-10-30,Orlando Magic,0,1,47.0,100.0,0.470,10.0,27.0,0.370,11.0,18.0,14.0,40.0,21.0,29.0,7.0,18.0,6.0,115.0,10.5,-5.0,O,199.5
7,21300010,2013-10-30,Minnesota Timberwolves,1,0,42.0,100.0,0.420,8.0,25.0,0.320,28.0,34.0,14.0,35.0,27.0,19.0,7.0,14.0,3.0,120.0,-10.5,5.0,O,199.5
8,21300008,2013-10-30,New York Knicks,1,0,36.0,71.0,0.507,3.0,13.0,0.231,15.0,17.0,6.0,28.0,17.0,16.0,12.0,22.0,6.0,90.0,-7.5,7.0,U,196.5
9,21300008,2013-10-30,Milwaukee Bucks,0,1,34.0,74.0,0.459,5.0,16.0,0.313,10.0,12.0,5.0,26.0,19.0,22.0,10.0,23.0,4.0,83.0,7.5,-7.0,U,196.5


In [239]:
sort_df = df.sort_values(by=['team','date'])
display(sort_df)

,GAME_ID,date,team,home,away,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,AST,PF,STL,TO,BLK,PTS,spread,PLUS_MINUS,ou,total
19,21300012,2013-10-30,Atlanta Hawks,0,1,37.0,76.0,0.487,8.0,24.0,0.333,27.0,35.0,5.0,28.0,27.0,20.0,16.0,17.0,5.0,109.0,6.0,-9.0,O,196.0
45,21300023,2013-11-01,Atlanta Hawks,1,0,36.0,77.0,0.468,10.0,23.0,0.435,20.0,31.0,7.0,30.0,24.0,12.0,7.0,13.0,3.0,102.0,-3.5,7.0,O,196.0
81,21300046,2013-11-03,Atlanta Hawks,0,1,40.0,90.0,0.444,9.0,22.0,0.409,14.0,26.0,10.0,42.0,26.0,22.0,9.0,11.0,3.0,103.0,-1.0,-2.0,O,205.0
107,21300059,2013-11-05,Atlanta Hawks,0,1,39.0,81.0,0.481,8.0,22.0,0.364,19.0,26.0,9.0,27.0,26.0,15.0,11.0,14.0,7.0,105.0,2.0,5.0,O,200.0
143,21300071,2013-11-07,Atlanta Hawks,0,1,43.0,94.0,0.457,8.0,26.0,0.308,13.0,19.0,13.0,33.0,31.0,23.0,11.0,11.0,9.0,107.0,4.5,-2.0,O,204.0
169,21300086,2013-11-09,Atlanta Hawks,1,0,43.0,90.0,0.478,6.0,19.0,0.316,12.0,17.0,7.0,32.0,36.0,15.0,9.0,12.0,6.0,104.0,-6.5,10.0,U,199.0
207,21300098,2013-11-11,Atlanta Hawks,0,1,35.0,76.0,0.461,10.0,27.0,0.370,23.0,28.0,6.0,35.0,26.0,22.0,7.0,13.0,3.0,103.0,-2.0,9.0,O,196.0
227,21300116,2013-11-13,Atlanta Hawks,1,0,35.0,77.0,0.455,5.0,24.0,0.208,16.0,21.0,8.0,39.0,22.0,15.0,1.0,15.0,6.0,91.0,-4.5,-4.0,U,201.5
253,21300127,2013-11-15,Atlanta Hawks,1,0,39.0,81.0,0.481,10.0,23.0,0.435,25.0,31.0,6.0,37.0,25.0,23.0,6.0,15.0,5.0,113.0,-8.5,10.0,O,208.0
277,21300139,2013-11-16,Atlanta Hawks,0,1,44.0,78.0,0.564,9.0,22.0,0.409,13.0,17.0,5.0,31.0,20.0,15.0,9.0,13.0,4.0,110.0,4.0,20.0,U,202.5


In [240]:
dates = sort_df['date'].values
teams = sort_df['team'].values
team_names = np.unique(teams)
team_idxs = []

for team in team_names:
    idxs = np.where(teams==team)[0]
    team_idxs.append([np.min(idxs),np.max(idxs)])
rest_days = np.array([])
for team_idx in team_idxs:
    date_range = dates[team_idx[0]:team_idx[1]+1]
    deltas = np.zeros(len(date_range))
    for i in range(1,len(date_range)):
        cur_date = parser.parse(date_range[i])
        last_date = parser.parse(date_range[i-1])
        deltas[i] = (cur_date-last_date).days - 1
    rest_days = np.append(rest_days,deltas)
print(rest_days.shape,rest_days)    
print(teams.shape)
sort_df['rest'] = rest_days
    
    

(13084,) [0. 1. 1. ... 1. 2. 1.]
(13084,)


In [241]:
teams = sort_df['team'].values
unique_teams = np.unique(teams)
gameIDs = sort_df['GAME_ID'].values
arr_holder = np.zeros((sort_df.shape[0],sort_df.shape[1]),dtype='object')
for idx,gameID in enumerate(gameIDs):
    team = teams[idx]
    matchRows = np.where(gameIDs == gameID)
    matchRows = np.array(matchRows)
    newRow = matchRows[np.where(teams[matchRows] != team)]
    newDf = sort_df.iloc[newRow]
    arr_holder[idx,:] = newDf.values
cols = newDf.columns.values
new_cols = [col+'_A' for col in cols]

total_cols = np.append(cols,new_cols)

ess_arr = sort_df.values

total_arr = np.concatenate([ess_arr,arr_holder],axis=1)
total_df = pd.DataFrame(total_arr)
total_df.columns = total_cols
display(total_df)

,GAME_ID,date,team,home,away,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,AST,PF,STL,TO,BLK,PTS,spread,PLUS_MINUS,ou,total,rest,GAME_ID_A,date_A,team_A,home_A,away_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,OREB_A,DREB_A,AST_A,PF_A,STL_A,TO_A,BLK_A,PTS_A,spread_A,PLUS_MINUS_A,ou_A,total_A,rest_A
0,21300012,2013-10-30,Atlanta Hawks,0,1,37,76,0.487,8,24,0.333,27,35,5,28,27,20,16,17,5,109,6,-9,O,196,0,21300012,2013-10-30,Dallas Mavericks,1,0,44,77,0.571,11,24,0.458,19,24,7,35,31,22,7,20,2,118,-6,9,O,196,0
1,21300023,2013-11-01,Atlanta Hawks,1,0,36,77,0.468,10,23,0.435,20,31,7,30,24,12,7,13,3,102,-3.5,7,O,196,1,21300023,2013-11-01,Toronto Raptors,0,1,40,88,0.455,7,23,0.304,8,14,10,32,15,25,5,12,6,95,3.5,-7,O,196,1
2,21300046,2013-11-03,Atlanta Hawks,0,1,40,90,0.444,9,22,0.409,14,26,10,42,26,22,9,11,3,103,-1,-2,O,205,1,21300046,2013-11-03,Los Angeles Lakers,1,0,38,89,0.427,9,25,0.36,20,24,6,47,27,24,5,13,7,105,1,2,O,205,1
3,21300059,2013-11-05,Atlanta Hawks,0,1,39,81,0.481,8,22,0.364,19,26,9,27,26,15,11,14,7,105,2,5,O,200,1,21300059,2013-11-05,Sacramento Kings,1,0,35,80,0.438,10,31,0.323,20,21,10,29,22,26,10,18,4,100,-2,-5,O,200,2
4,21300071,2013-11-07,Atlanta Hawks,0,1,43,94,0.457,8,26,0.308,13,19,13,33,31,23,11,11,9,107,4.5,-2,O,204,1,21300071,2013-11-07,Denver Nuggets,1,0,42,86,0.488,14,25,0.56,11,17,10,35,28,18,4,16,8,109,-4.5,2,O,204,1
5,21300086,2013-11-09,Atlanta Hawks,1,0,43,90,0.478,6,19,0.316,12,17,7,32,36,15,9,12,6,104,-6.5,10,U,199,1,21300086,2013-11-09,Orlando Magic,0,1,38,82,0.463,8,17,0.471,10,18,9,27,22,24,9,17,2,94,6.5,-10,U,199,0
6,21300098,2013-11-11,Atlanta Hawks,0,1,35,76,0.461,10,27,0.37,23,28,6,35,26,22,7,13,3,103,-2,9,O,196,1,21300098,2013-11-11,Charlotte Hornets,1,0,33,81,0.407,5,13,0.385,23,29,10,30,22,24,4,14,0,94,2,-9,O,196,2
7,21300116,2013-11-13,Atlanta Hawks,1,0,35,77,0.455,5,24,0.208,16,21,8,39,22,15,1,15,6,91,-4.5,-4,U,201.5,1,21300116,2013-11-13,New York Knicks,0,1,37,88,0.42,12,34,0.353,9,13,8,29,24,19,11,3,3,95,4.5,4,U,201.5,2
8,21300127,2013-11-15,Atlanta Hawks,1,0,39,81,0.481,10,23,0.435,25,31,6,37,25,23,6,15,5,113,-8.5,10,O,208,1,21300127,2013-11-15,Philadelphia 76ers,0,1,40,86,0.465,5,19,0.263,18,23,9,30,21,19,6,16,4,103,8.5,-10,O,208,1
9,21300139,2013-11-16,Atlanta Hawks,0,1,44,78,0.564,9,22,0.409,13,17,5,31,20,15,9,13,4,110,4,20,U,202.5,0,21300139,2013-11-16,New York Knicks,1,0,34,90,0.378,10,30,0.333,12,16,19,29,19,20,6,14,3,90,-4,-20,U,202.5,1


In [242]:
len(total_df)/2

6542.0

In [243]:
games_back = 15
if games_back ==np.inf:
    gameCheck = 10
else:
    gameCheck = games_back
arr_holderSingle = np.zeros((sort_df.shape[0],sort_df.shape[1]+1),dtype=object)
arr_holder = np.zeros((sort_df.shape[0],sort_df.shape[1]+1),dtype=object)
arr_h_count = 0
data_cols = ['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','OREB','DREB','AST','PF','STL','TO','BLK','PTS']
badGameIDs = []
for team_idx in team_idxs:
    teamDf = sort_df[team_idx[0]:team_idx[1]+1]
    dates = teamDf['date'].values
    dateTimes = np.array([parser.parse(date) for date in dates])
    dayDiff = [dT.days for dT in np.diff(dateTimes)]
    dayDiff.insert(0,400) #first entry is always new season
    dayDiff = np.array(dayDiff)
    seasIdxs = np.where(dayDiff > 100)[0]
    for Ridx in range(len(teamDf)):
        curSeas = np.max(np.where(seasIdxs <= Ridx)[0])
        SeasSt = seasIdxs[curSeas]

        if Ridx != SeasSt and Ridx-gameCheck > SeasSt:
            #not starting a new season
            startIdx = Ridx - games_back
            if startIdx < SeasSt:
                startIdx = SeasSt
            subDf = teamDf[startIdx:Ridx]
            
            data = subDf[data_cols].values
            data_mean = np.mean(data,axis=0)
            RowDf = teamDf.iloc[[Ridx]]
            Row_data = RowDf.values
            Row_data[:,5:21] = data_mean
            arr_holder[arr_h_count,:-1] = Row_data
            arr_holder[arr_h_count,-1] = RowDf['PTS'].values[0]
            
            arr_holderSingle[arr_h_count,:-1] = RowDf.values
            arr_holderSingle[arr_h_count,-1] = RowDf['PTS'].values[0]
            arr_h_count+=1
        else:
            badGameIDs.append(teamDf.iloc[[Ridx]]['GAME_ID'].values[0])
            
            

        
print(seasIdxs)
print(dates[seasIdxs])

[  0  93 185 267 362]
['2013-10-30' '2014-10-29' '2015-10-28' '2016-10-27' '2017-10-18']


In [244]:
arr_holder = arr_holder[:-len(badGameIDs),:]
arr_holderSingle = arr_holderSingle[:-len(badGameIDs),:]

In [245]:
cols = sort_df.columns.values
for cIdx,col in enumerate(cols):
    if col in data_cols:
        cols[cIdx] = col+'_Ravg'
cols = np.append(cols,'PTS_G')
avg_df = pd.DataFrame(arr_holder,columns=cols)
display(avg_df)
single_df = pd.DataFrame(arr_holderSingle,columns=cols)

,GAME_ID,date,team,home,away,FGM_Ravg,FGA_Ravg,FG_PCT_Ravg,FG3M_Ravg,FG3A_Ravg,FG3_PCT_Ravg,FTM_Ravg,FTA_Ravg,OREB_Ravg,DREB_Ravg,AST_Ravg,PF_Ravg,STL_Ravg,TO_Ravg,BLK_Ravg,PTS_Ravg,spread,PLUS_MINUS,ou,total,rest,PTS_G
0,21300231,2013-11-29,Atlanta Hawks,1,0,37.2,81,0.460133,8,22.2667,0.356667,16.1333,22,8.13333,32.2,24.4667,18.5333,8.13333,14.4,4.73333,98.5333,1.5,1,U,202,1,88
1,21300241,2013-11-30,Atlanta Hawks,0,1,36.8667,80.7333,0.457267,7.53333,21.7333,0.341,16.3333,21.9333,8.26667,32.4667,24.1333,18.8,8.33333,14.6,4.8,97.6,4.5,-7,O,190,0,101
2,21300258,2013-12-02,Atlanta Hawks,0,1,36.6,80.1333,0.457267,7.4,21.4,0.3412,16.8667,21.9333,8.2,31.7333,23.6667,18.5333,8.2,14.9333,5,97.4667,12,-2,O,199,1,100
3,21300270,2013-12-04,Atlanta Hawks,1,0,36.8,81.1333,0.4544,7.6,22,0.3406,15.9333,20.5333,8.26667,31.8667,23.4,18.8,8.13333,14.6667,4.66667,97.1333,2.5,10,P,204,1,107
4,21300282,2013-12-06,Atlanta Hawks,1,0,36.7333,80.3333,0.458067,7.86667,21.8,0.354867,15.8,20.3333,7.93333,31.8,23.2,18.7333,7.66667,14.6667,4.33333,97.1333,-6,19,O,196.5,1,108
5,21300313,2013-12-10,Atlanta Hawks,1,0,37,80.4,0.4606,8,22.0667,0.357,15.4,19.9333,8.4,32.0667,22.8667,18.9333,7.46667,14.6667,4.13333,97.4,3.5,-9,U,204,3,92
6,21300332,2013-12-13,Atlanta Hawks,1,0,36.7333,81.1333,0.4536,7.93333,22,0.3554,15.2667,19.8667,8.8,31.9333,22.7333,18.9333,7.66667,14.8,4.26667,96.6667,-6,2,O,194,2,101
7,21300345,2013-12-14,Atlanta Hawks,0,1,36.9333,81.9333,0.451733,8.06667,21.8,0.363733,15.4,20.0667,9,31.6667,23.0667,18.8667,8.26667,14.8,3.93333,97.3333,-1,-5,O,192.5,0,106
8,21300357,2013-12-16,Atlanta Hawks,1,0,37,80.9333,0.460067,8.4,22.4,0.366,14.4667,18.7333,9,31.0667,22.8667,18.9333,8.26667,15.6,3.66667,96.8667,-6,14,O,204.5,1,114
9,21300372,2013-12-18,Atlanta Hawks,1,0,37.0667,81.8667,0.455067,8.53333,22.7333,0.365867,14.4667,18.6,9.66667,30.9333,23.8,19.1333,8.33333,15.4,3.8,97.1333,-6,17,O,205.5,1,124


In [246]:
teams = avg_df['team'].values
unique_teams = np.unique(teams)
gameIDs = avg_df['GAME_ID'].values
arr_holder = np.zeros((avg_df.shape[0],avg_df.shape[1]),dtype='object')
arr_holderSingle = np.zeros((avg_df.shape[0],avg_df.shape[1]),dtype='object')
for idx,gameID in enumerate(gameIDs):
    if gameID not in badGameIDs:
        team = teams[idx]
        matchRows = np.where(gameIDs == gameID)
        matchRows = np.array(matchRows)
        newRow = matchRows[np.where(teams[matchRows] != team)]
        newDf = avg_df.iloc[newRow]
        arr_holder[idx,:] = newDf.values
        newDfSingle = single_df.iloc[newRow]
        arr_holderSingle[idx,:] = newDfSingle.values
cols = newDf.columns.values
new_cols = [col+'_A' for col in cols]

total_cols = np.append(cols,new_cols)

ess_arr = avg_df.values
ess_arrSingle = single_df.values

total_arr = np.concatenate([ess_arr,arr_holder],axis=1)
total_df = pd.DataFrame(total_arr)
total_df.columns = total_cols
total_arr_Single = np.concatenate([ess_arrSingle,arr_holderSingle],axis=1)
total_dfSingle = pd.DataFrame(total_arr_Single,columns=total_cols)
display(total_df)

,GAME_ID,date,team,home,away,FGM_Ravg,FGA_Ravg,FG_PCT_Ravg,FG3M_Ravg,FG3A_Ravg,FG3_PCT_Ravg,FTM_Ravg,FTA_Ravg,OREB_Ravg,DREB_Ravg,AST_Ravg,PF_Ravg,STL_Ravg,TO_Ravg,BLK_Ravg,PTS_Ravg,spread,PLUS_MINUS,ou,total,rest,PTS_G,GAME_ID_A,date_A,team_A,home_A,away_A,FGM_Ravg_A,FGA_Ravg_A,FG_PCT_Ravg_A,FG3M_Ravg_A,FG3A_Ravg_A,FG3_PCT_Ravg_A,FTM_Ravg_A,FTA_Ravg_A,OREB_Ravg_A,DREB_Ravg_A,AST_Ravg_A,PF_Ravg_A,STL_Ravg_A,TO_Ravg_A,BLK_Ravg_A,PTS_Ravg_A,spread_A,PLUS_MINUS_A,ou_A,total_A,rest_A,PTS_G_A
0,21300231,2013-11-29,Atlanta Hawks,1,0,37.2,81,0.460133,8,22.2667,0.356667,16.1333,22,8.13333,32.2,24.4667,18.5333,8.13333,14.4,4.73333,98.5333,1.5,1,U,202,1,88,21300231,2013-11-29,Dallas Mavericks,0,1,38.9333,84.1333,0.4648,8.8,22.9333,0.382333,18,22.2,10.5333,30.4667,22.2667,20.9333,9.6,14.8,4.53333,104.667,-1.5,-1,U,202,1,87
1,21300241,2013-11-30,Atlanta Hawks,0,1,36.8667,80.7333,0.457267,7.53333,21.7333,0.341,16.3333,21.9333,8.26667,32.4667,24.1333,18.8,8.33333,14.6,4.8,97.6,4.5,-7,O,190,0,101,21300241,2013-11-30,Washington Wizards,1,0,37.8,84.9333,0.446,8.73333,22.8667,0.376933,14,18.9333,10.3333,31.0667,23.9333,17.6,9.4,14.6,4.4,98.3333,-4.5,7,O,190,0,108
2,21300258,2013-12-02,Atlanta Hawks,0,1,36.6,80.1333,0.457267,7.4,21.4,0.3412,16.8667,21.9333,8.2,31.7333,23.6667,18.5333,8.2,14.9333,5,97.4667,12,-2,O,199,1,100,21300258,2013-12-02,San Antonio Spurs,1,0,40.7333,82.8,0.4932,8.2,20.8667,0.389867,13.0667,17.2,8.2,33.8667,25,17.2667,8.8,14.6,4.86667,102.733,-12,2,O,199,1,102
3,21300270,2013-12-04,Atlanta Hawks,1,0,36.8,81.1333,0.4544,7.6,22,0.3406,15.9333,20.5333,8.26667,31.8667,23.4,18.8,8.13333,14.6667,4.66667,97.1333,2.5,10,P,204,1,107,21300270,2013-12-04,Los Angeles Clippers,0,1,38.9333,82.7333,0.472067,7.8,23.5333,0.3252,19.4667,27.4,10.6667,33.1333,24.2667,22.8,8.06667,13.8667,4.46667,105.133,-2.5,-10,P,204,2,97
4,21300282,2013-12-06,Atlanta Hawks,1,0,36.7333,80.3333,0.458067,7.86667,21.8,0.354867,15.8,20.3333,7.93333,31.8,23.2,18.7333,7.66667,14.6667,4.33333,97.1333,-6,19,O,196.5,1,108,21300282,2013-12-06,Cleveland Cavaliers,0,1,35.6667,83.9333,0.425733,7.33333,20.0667,0.358,15.6,20.4667,11.8,31.2667,19,19.2,7.8,15.6667,5.53333,94.2667,6,-19,O,196.5,1,89
5,21300313,2013-12-10,Atlanta Hawks,1,0,37,80.4,0.4606,8,22.0667,0.357,15.4,19.9333,8.4,32.0667,22.8667,18.9333,7.46667,14.6667,4.13333,97.4,3.5,-9,U,204,3,92,21300313,2013-12-10,Oklahoma City Thunder,0,1,39.2,82.8667,0.475467,6.46667,17.8,0.362467,21.6667,26.4667,11.2667,36.4667,21.6667,21.6667,7.33333,15.3333,6.8,106.533,-3.5,9,U,204,1,101
6,21300332,2013-12-13,Atlanta Hawks,1,0,36.7333,81.1333,0.4536,7.93333,22,0.3554,15.2667,19.8667,8.8,31.9333,22.7333,18.9333,7.66667,14.8,4.26667,96.6667,-6,2,O,194,2,101,21300332,2013-12-13,Washington Wizards,0,1,36.2,82.3333,0.441267,7.73333,21.2,0.376133,15.6667,21.2,9.86667,32.1333,22.6,17.8,9.2,14.4,5,95.8,6,-2,O,194,3,99
7,21300345,2013-12-14,Atlanta Hawks,0,1,36.9333,81.9333,0.451733,8.06667,21.8,0.363733,15.4,20.0667,9,31.6667,23.0667,18.8667,8.26667,14.8,3.93333,97.3333,-1,-5,O,192.5,0,106,21300345,2013-12-14,New York Knicks,1,0,35.4667,82.5333,0.4306,8.66667,24.8667,0.341,13.8667,17.6,10.5333,28.9333,20.2,22.8667,7.93333,11.8667,4.73333,93.4667,1,5,O,192.5,0,111
8,21300357,2013-12-16,Atlanta Hawks,1,0,37,80.9333,0.460067,8.4,22.4,0.366,14.4667,18.7333,9,31.0667,22.8667,18.9333,8.26667,15.6,3.66667,96.8667,-6,14,O,204.5,1,114,21300357,2013-12-16,Los Angeles Lakers,0,1,38.4,83.7333,0.459,9.86667,25.4,0.387267,15.6,20.8,9.6,31.6667,24.3333,20.6667,7.26667,13.9333,6.06667,102.267,6,-14,O,204.5,1,100
9,21300372,2013-12-18,Atlanta Hawks,1,0,37.0667,81.8667,0.455067,8.53333,22.7333,0.365867,14.4667,18.6,9.66667,30.9333,23.8,19.1333,8.33333,15.4,3.8,97.1333,-6,17,O,205.5,1,124,21300372,2013-12-18,Sacramento Kings,0,1,36.9333,84.3333,0.439067,6.6,19.7333,0.328267,21.2,26.6,11.8,31.9333,21,23.4,8.33333,14.0667,3.86667,101.667,6,-17,O,205.5,0,107


In [270]:
#treating all teams identically
cols2drop = ['GAME_ID','date','PLUS_MINUS','total','spread','ou','PTS_G']
cols2drop = cols2drop + [col+'_A' for col in cols2drop]
cols2drop.append('home_A')
cols2drop.append('away_A')


DataDf = total_df.drop(cols2drop,axis=1,inplace=False)
diff = total_df['PTS_G'].values-total_df['PTS_G_A'].values
#DataDf['Diff'] = diff
spread = total_df['spread'].values
DataDf['spread'] = spread
covered = np.zeros(len(spread))
covered[np.where(spread+diff >= 0 )]= 1
DataDf['Covered?'] = covered
DataDfSingle = total_dfSingle.drop(cols2drop,axis=1,inplace=False)
diffSingle = total_dfSingle['PTS_G'].values-total_dfSingle['PTS_G_A'].values
#DataDf['Diff'] = diff
spread = total_df['spread'].values
DataDf['spread'] = spread
DataDfSingle['spread'] = spread
covered = np.zeros(len(spread))
covered[np.where(spread+diff >= 0 )]= 1
DataDf['Covered?'] = covered
DataDfSingle['Covered?'] = covered

good_cols = ['home','away','Diff']
scaler = sklearn.preprocessing.MinMaxScaler()
'''
for key in DataDf.columns:
    if key not in good_cols:
        DataDf[key] = scaler.fit_transform(DataDf[key].values.reshape(-1,1))
        DataDfSingle[key] = scaler.fit_transform(DataDfSingle[key].values.reshape(-1,1))
'''
bad_cols = ['FG_PCT_Ravg','FG3_PCT_Ravg','FG_PCT_Ravg_A','FG3_PCT_Ravg_A']
DataDf.drop(bad_cols,axis=1,inplace=True)
display(DataDf)
DataDfSingle.drop(bad_cols,axis=1,inplace=True)

,team,home,away,FGM_Ravg,FGA_Ravg,FG3M_Ravg,FG3A_Ravg,FTM_Ravg,FTA_Ravg,OREB_Ravg,DREB_Ravg,AST_Ravg,PF_Ravg,STL_Ravg,TO_Ravg,BLK_Ravg,PTS_Ravg,rest,team_A,FGM_Ravg_A,FGA_Ravg_A,FG3M_Ravg_A,FG3A_Ravg_A,FTM_Ravg_A,FTA_Ravg_A,OREB_Ravg_A,DREB_Ravg_A,AST_Ravg_A,PF_Ravg_A,STL_Ravg_A,TO_Ravg_A,BLK_Ravg_A,PTS_Ravg_A,rest_A,spread,Covered?
0,Atlanta Hawks,1,0,37.2,81,8,22.2667,16.1333,22,8.13333,32.2,24.4667,18.5333,8.13333,14.4,4.73333,98.5333,1,Dallas Mavericks,38.9333,84.1333,8.8,22.9333,18,22.2,10.5333,30.4667,22.2667,20.9333,9.6,14.8,4.53333,104.667,1,1.5,1.0
1,Atlanta Hawks,0,1,36.8667,80.7333,7.53333,21.7333,16.3333,21.9333,8.26667,32.4667,24.1333,18.8,8.33333,14.6,4.8,97.6,0,Washington Wizards,37.8,84.9333,8.73333,22.8667,14,18.9333,10.3333,31.0667,23.9333,17.6,9.4,14.6,4.4,98.3333,0,4.5,0.0
2,Atlanta Hawks,0,1,36.6,80.1333,7.4,21.4,16.8667,21.9333,8.2,31.7333,23.6667,18.5333,8.2,14.9333,5,97.4667,1,San Antonio Spurs,40.7333,82.8,8.2,20.8667,13.0667,17.2,8.2,33.8667,25,17.2667,8.8,14.6,4.86667,102.733,1,12,1.0
3,Atlanta Hawks,1,0,36.8,81.1333,7.6,22,15.9333,20.5333,8.26667,31.8667,23.4,18.8,8.13333,14.6667,4.66667,97.1333,1,Los Angeles Clippers,38.9333,82.7333,7.8,23.5333,19.4667,27.4,10.6667,33.1333,24.2667,22.8,8.06667,13.8667,4.46667,105.133,2,2.5,1.0
4,Atlanta Hawks,1,0,36.7333,80.3333,7.86667,21.8,15.8,20.3333,7.93333,31.8,23.2,18.7333,7.66667,14.6667,4.33333,97.1333,1,Cleveland Cavaliers,35.6667,83.9333,7.33333,20.0667,15.6,20.4667,11.8,31.2667,19,19.2,7.8,15.6667,5.53333,94.2667,1,-6,1.0
5,Atlanta Hawks,1,0,37,80.4,8,22.0667,15.4,19.9333,8.4,32.0667,22.8667,18.9333,7.46667,14.6667,4.13333,97.4,3,Oklahoma City Thunder,39.2,82.8667,6.46667,17.8,21.6667,26.4667,11.2667,36.4667,21.6667,21.6667,7.33333,15.3333,6.8,106.533,1,3.5,0.0
6,Atlanta Hawks,1,0,36.7333,81.1333,7.93333,22,15.2667,19.8667,8.8,31.9333,22.7333,18.9333,7.66667,14.8,4.26667,96.6667,2,Washington Wizards,36.2,82.3333,7.73333,21.2,15.6667,21.2,9.86667,32.1333,22.6,17.8,9.2,14.4,5,95.8,3,-6,0.0
7,Atlanta Hawks,0,1,36.9333,81.9333,8.06667,21.8,15.4,20.0667,9,31.6667,23.0667,18.8667,8.26667,14.8,3.93333,97.3333,0,New York Knicks,35.4667,82.5333,8.66667,24.8667,13.8667,17.6,10.5333,28.9333,20.2,22.8667,7.93333,11.8667,4.73333,93.4667,0,-1,0.0
8,Atlanta Hawks,1,0,37,80.9333,8.4,22.4,14.4667,18.7333,9,31.0667,22.8667,18.9333,8.26667,15.6,3.66667,96.8667,1,Los Angeles Lakers,38.4,83.7333,9.86667,25.4,15.6,20.8,9.6,31.6667,24.3333,20.6667,7.26667,13.9333,6.06667,102.267,1,-6,1.0
9,Atlanta Hawks,1,0,37.0667,81.8667,8.53333,22.7333,14.4667,18.6,9.66667,30.9333,23.8,19.1333,8.33333,15.4,3.8,97.1333,1,Sacramento Kings,36.9333,84.3333,6.6,19.7333,21.2,26.6,11.8,31.9333,21,23.4,8.33333,14.0667,3.86667,101.667,0,-6,1.0


In [282]:
team_names = DataDf['team'].values
team_names_A = DataDf['team'].values
idx_Split = len(team_names)
team_names_Tot = np.append(team_names,team_names_A)
le = LabelEncoder()
le.fit(team_names_Tot)

team_namesT = le.transform(team_names_Tot)
DataDf['team'] = team_namesT[:idx_Split]
DataDf['team_A'] = team_namesT[idx_Split:]

DataDfSingle['team'] = team_namesT[:idx_Split]
DataDfSingle['team_A'] = team_namesT[idx_Split:]

In [283]:

DataDf = pd.get_dummies(DataDf,columns=['team','team_A','Covered?'])
DataDfSingle = pd.get_dummies(DataDfSingle,columns=['team','team_A','Covered?'])
display(DataDf)

,home,away,FGM_Ravg,FGA_Ravg,FG3M_Ravg,FG3A_Ravg,FTM_Ravg,FTA_Ravg,OREB_Ravg,DREB_Ravg,AST_Ravg,PF_Ravg,STL_Ravg,TO_Ravg,BLK_Ravg,PTS_Ravg,rest,FGM_Ravg_A,FGA_Ravg_A,FG3M_Ravg_A,FG3A_Ravg_A,FTM_Ravg_A,FTA_Ravg_A,OREB_Ravg_A,DREB_Ravg_A,AST_Ravg_A,PF_Ravg_A,STL_Ravg_A,TO_Ravg_A,BLK_Ravg_A,PTS_Ravg_A,rest_A,spread,team_0,team_1,team_2,team_3,team_4,team_5,team_6,team_7,team_8,team_9,team_10,team_11,team_12,team_13,team_14,team_15,team_16,team_17,team_18,team_19,team_20,team_21,team_22,team_23,team_24,team_25,team_26,team_27,team_28,team_29,team_A_0,team_A_1,team_A_2,team_A_3,team_A_4,team_A_5,team_A_6,team_A_7,team_A_8,team_A_9,team_A_10,team_A_11,team_A_12,team_A_13,team_A_14,team_A_15,team_A_16,team_A_17,team_A_18,team_A_19,team_A_20,team_A_21,team_A_22,team_A_23,team_A_24,team_A_25,team_A_26,team_A_27,team_A_28,team_A_29,Covered?_0.0,Covered?_1.0
0,1,0,37.2,81,8,22.2667,16.1333,22,8.13333,32.2,24.4667,18.5333,8.13333,14.4,4.73333,98.5333,1,38.9333,84.1333,8.8,22.9333,18,22.2,10.5333,30.4667,22.2667,20.9333,9.6,14.8,4.53333,104.667,1,1.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,36.8667,80.7333,7.53333,21.7333,16.3333,21.9333,8.26667,32.4667,24.1333,18.8,8.33333,14.6,4.8,97.6,0,37.8,84.9333,8.73333,22.8667,14,18.9333,10.3333,31.0667,23.9333,17.6,9.4,14.6,4.4,98.3333,0,4.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,1,36.6,80.1333,7.4,21.4,16.8667,21.9333,8.2,31.7333,23.6667,18.5333,8.2,14.9333,5,97.4667,1,40.7333,82.8,8.2,20.8667,13.0667,17.2,8.2,33.8667,25,17.2667,8.8,14.6,4.86667,102.733,1,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,0,36.8,81.1333,7.6,22,15.9333,20.5333,8.26667,31.8667,23.4,18.8,8.13333,14.6667,4.66667,97.1333,1,38.9333,82.7333,7.8,23.5333,19.4667,27.4,10.6667,33.1333,24.2667,22.8,8.06667,13.8667,4.46667,105.133,2,2.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,0,36.7333,80.3333,7.86667,21.8,15.8,20.3333,7.93333,31.8,23.2,18.7333,7.66667,14.6667,4.33333,97.1333,1,35.6667,83.9333,7.33333,20.0667,15.6,20.4667,11.8,31.2667,19,19.2,7.8,15.6667,5.53333,94.2667,1,-6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,1,0,37,80.4,8,22.0667,15.4,19.9333,8.4,32.0667,22.8667,18.9333,7.46667,14.6667,4.13333,97.4,3,39.2,82.8667,6.46667,17.8,21.6667,26.4667,11.2667,36.4667,21.6667,21.6667,7.33333,15.3333,6.8,106.533,1,3.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,1,0,36.7333,81.1333,7.93333,22,15.2667,19.8667,8.8,31.9333,22.7333,18.9333,7.66667,14.8,4.26667,96.6667,2,36.2,82.3333,7.73333,21.2,15.6667,21.2,9.86667,32.1333,22.6,17.8,9.2,14.4,5,95.8,3,-6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,0,1,36.9333,81.9333,8.06667,21.8,15.4,20.0667,9,31.6667,23.0667,18.8667,8.26667,14.8,3.93333,97.3333,0,35.4667,82.5333,8.66667,24.8667,13.8667,17.6,10.5333,28.9333,20.2,22.8667,7.93333,11.8667,4.73333,93.4667,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,1,0,37,80.9333,8.4,22.4,14.4667,18.7333,9,31.0667,22.8667,18.9333,8.26667,15.6,3.66667,96.8667,1,38.4,83.7333,9.86667,25.4,15.6,20.8,9.6,31.6667,24.3333,20.6667,7.26667,13.9333,6.06667,102.267,1,-6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,1,0,37.0667,81.8667,8.53333,22.7333,14.4667,18.6,9.66667,30.9333,23.8,19.1333,8.33333,15.4,3.8,97.1333,1,36.9333,84.3333,6.6,19.7333,21.2,26.6,11.8,31.9333,21,23.4,8.33333,14.0667,3.86667,101.667,0,-6,1,0,0,

In [284]:
print(diff,spread)

[1.0 -7.0 -2.0 ... 8.0 -10.0 -10.0] [1.5 4.5 12.0 ... 1.5 7.0 2.0]


In [285]:
#now lets make the model
test = 10
data = DataDf.values
dataSingle = DataDfSingle.values
test_size = .1

num_test = np.round(test_size*data.shape[0])
num_train = data.shape[0] - num_test
test_idxs = np.random.randint(0,int(data.shape[0]),int(num_test))
test_idxs = np.unique(test_idxs)
while test_idxs.shape[0] != num_test:
    test_idxs = np.append(test_idxs, np.random.randint(0,int(data.shape[0])))
    test_idxs = np.unique(test_idxs)                      
                          
train_idxs = np.array([i for i in range(data.shape[0]) if i not in test_idxs])
    


x = data[:,:-2]
y = data[:,-2:]
y = y.astype('int')
x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size=.1)

In [286]:
x.shape

(10684, 93)

In [287]:
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
model = basemodel()
#model.fit(x_train,y_train,epochs=100,batch_size=150)
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(30, activation='relu', input_dim=93))
model.add(Dropout(0.3))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=32)
score = model.evaluate(x_test, y_test, batch_size=32)


Epoch 1/20
9615/9615 [==============================] - 3s 262us/step - loss: 1.7446 - acc: 0.5010
Epoch 2/20
9615/9615 [==============================] - 1s 110us/step - loss: 0.6960 - acc: 0.5056
Epoch 3/20
9615/9615 [==============================] - 1s 104us/step - loss: 0.6893 - acc: 0.5098
Epoch 4/20
9615/9615 [==============================] - 1s 101us/step - loss: 0.6896 - acc: 0.5076
Epoch 5/20
9615/9615 [==============================] - 1s 103us/step - loss: 0.6885 - acc: 0.5108
Epoch 6/20
9615/9615 [==============================] - 1s 100us/step - loss: 0.6882 - acc: 0.5122
Epoch 7/20
9615/9615 [==============================] - 1s 107us/step - loss: 0.6886 - acc: 0.5111
Epoch 8/20
9615/9615 [==============================] - 1s 105us/step - loss: 0.6883 - acc: 0.5113
Epoch 9/20
9615/9615 [==============================] - 1s 109us/step - loss: 0.6883 - acc: 0.5112
Epoch 10/20
9615/9615 [==============================] - 1s 110us/step - loss: 0.6886 - acc: 0.5126 0s - loss

In [288]:
print(score,model.metrics_names)

[0.6842267422948183, 0.5285313377266626] ['loss', 'acc']


In [289]:
accs = []
for i in range(10):
    x = data[:,:-2]
    y = data[:,-2:]
    y = y.astype('int')
    x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size=.1)
    clf = RandomForestClassifier(n_estimators=20)
    clf.fit(x_train,y_train)
    acc = clf.score(x_test,y_test)
    print(acc)
    accs.append(acc)
print(np.mean(accs))

0.4452759588400374
0.42656688493919553
0.42188961646398504
0.4303086997193639
0.4087932647333957
0.44340505144995324
0.43592142188961647
0.43405051449953225
0.42095416276894293
0.4097287184284378
0.427689429373246


In [290]:
data = DataDf.values
dataSingle = DataDfSingle.values
test_size = .1

num_test = np.round(test_size*data.shape[0])
num_train = data.shape[0] - num_test
test_idxs = np.random.randint(0,int(data.shape[0]),int(num_test))
test_idxs = np.unique(test_idxs)
while test_idxs.shape[0] != num_test:
    test_idxs = np.append(test_idxs, np.random.randint(0,int(data.shape[0])))
    test_idxs = np.unique(test_idxs)                      
                          
train_idxs = np.array([i for i in range(data.shape[0]) if i not in test_idxs])
    
num_test = np.round(test_size*data.shape[0])
num_train = data.shape[0] - num_test
test_idxs = np.random.randint(0,int(data.shape[0]),int(num_test))
test_idxs = np.unique(test_idxs)
while test_idxs.shape[0] != num_test:
    test_idxs = np.append(test_idxs, np.random.randint(0,int(data.shape[0])))
    test_idxs = np.unique(test_idxs)                      
                          
train_idxs = np.array([i for i in range(data.shape[0]) if i not in test_idxs])



x_train = dataSingle[train_idxs,:-2]
x_test = data[test_idxs,:-2]
y_train = dataSingle[train_idxs,-2:]
y_test = data[test_idxs,-2:]

In [291]:
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
model = basemodel()
#model.fit(x_train,y_train,epochs=100,batch_size=150)
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(30, activation='relu', input_dim=93))
model.add(Dropout(0.5))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=32)
score = model.evaluate(x_test, y_test, batch_size=32)


Epoch 1/20
9616/9616 [==============================] - 3s 266us/step - loss: 2.0833 - acc: 0.5462
Epoch 2/20
9616/9616 [==============================] - 1s 102us/step - loss: 0.6512 - acc: 0.6441
Epoch 3/20
9616/9616 [==============================] - 1s 103us/step - loss: 0.6017 - acc: 0.6797
Epoch 4/20
9616/9616 [==============================] - 1s 100us/step - loss: 0.5277 - acc: 0.7553
Epoch 5/20
9616/9616 [==============================] - 1s 109us/step - loss: 0.4506 - acc: 0.8039
Epoch 6/20
9616/9616 [==============================] - 1s 108us/step - loss: 0.4098 - acc: 0.8284
Epoch 7/20
9616/9616 [==============================] - 1s 104us/step - loss: 0.3875 - acc: 0.8389
Epoch 8/20
9616/9616 [==============================] - 1s 105us/step - loss: 0.3569 - acc: 0.8557
Epoch 9/20
9616/9616 [==============================] - 1s 103us/step - loss: 0.3312 - acc: 0.8677
Epoch 10/20
9616/9616 [==============================] - 1s 105us/step - loss: 0.3281 - acc: 0.8695
Epoch 11/

In [292]:
print(score,model.metrics_names)

[1.7372912543543269, 0.5205992507130912] ['loss', 'acc']


In [293]:
accs = []
for i in range(10):
    x = data[:,:-2]
    y = data[:,-2:]
    y = y.astype('int')
    x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size=.1)
    clf = RandomForestClassifier(n_estimators=20)
    clf.fit(x_train,y_train)
    acc = clf.score(x_test,y_test)
    print(acc)
    accs.append(acc)

0.4499532273152479
0.41347053320860616
0.43592142188961647
0.46211412535079516
0.44340505144995324
0.44340505144995324
0.43966323666978485
0.4284377923292797
0.4452759588400374
0.41347053320860616


In [206]:
np.mean(accs)

0.46975524475524477

In [255]:
y_train.shape[0]

9615